### importing libraries

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

### data cleaning

In [2]:
new=pd.read_csv("new_Portmap.csv",low_memory=False)

In [3]:
# drop SimillarHTTP because its causes unknown errors
new.drop("SimillarHTTP", axis=1, inplace=True)

transform object data to numeric data

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
 
# Using .fit_transform function to fit label
# encoder and return encoded label
c=0
columns=new.columns
for i in new.dtypes:
    if(i=="object"  ):
        label = le.fit_transform(new[columns[c]])
        new.drop(columns[c], axis=1, inplace=True)
        new[columns[c]] = label
    c+=1


handle missing values and infinite

In [6]:
nul=new.isnull().sum()
c=0
for i in nul:
    print(columns[c],end="\t")
    print(i)
    c+=1

Unnamed: 0.1	0
Unnamed: 0	0
Flow ID	0
Source IP	0
Source Port	0
Destination IP	0
Destination Port	0
Protocol	0
Timestamp	0
Flow Duration	0
Total Fwd Packets	0
Total Backward Packets	0
Total Length of Fwd Packets	0
Total Length of Bwd Packets	0
Fwd Packet Length Max	0
Fwd Packet Length Min	0
Fwd Packet Length Mean	0
Fwd Packet Length Std	0
Bwd Packet Length Max	0
Bwd Packet Length Min	0
Bwd Packet Length Mean	0
Bwd Packet Length Std	0
Flow Bytes/s	0
Flow Packets/s	0
Flow IAT Mean	0
Flow IAT Std	0
Flow IAT Max	0
Flow IAT Min	0
Fwd IAT Total	0
Fwd IAT Mean	0
Fwd IAT Std	0
Fwd IAT Max	0
Fwd IAT Min	0
Bwd IAT Total	0
Bwd IAT Mean	0
Bwd IAT Std	0
Bwd IAT Max	0
Bwd IAT Min	0
Fwd PSH Flags	0
Bwd PSH Flags	0
Fwd URG Flags	0
Bwd URG Flags	0
Fwd Header Length	0
Bwd Header Length	0
Fwd Packets/s	0
Bwd Packets/s	0
Min Packet Length	0
Max Packet Length	0
Packet Length Mean	0
Packet Length Std	0
Packet Length Variance	0
FIN Flag Count	0
SYN Flag Count	0
RST Flag Count	0
PSH Flag Count	0
ACK Flag Coun

In [6]:
print(np.any(np.isnan(new)))
print(np.all(np.isfinite(new)))

False
True


In [5]:
new = new.dropna(axis=1)
new = new.reset_index()
target="Label"
new = new[np.isfinite(new).all(1)]
X=new.drop([target],axis=1)
y=new[target]

check data types 

In [ ]:
c=0
columns=new.columns
for i in new.dtypes:
    print(c,end=" ")
    print(columns[c],end="\t")
    print(i)
    c+=1

In [8]:
print(X.shape)
print(y.shape)

(181894, 87)
(181894,)


### Feature Selection

#### RFFI

In [ ]:
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeRegressor

dtree = DecisionTreeRegressor()
rfecv = RFECV(estimator=dtree, step=1, scoring="neg_mean_squared_error", cv=4, verbose=1,
                  n_jobs = 4)
rfecv.fit(X, y)
rfecv.transform(X)
print(rfecv)
print(rfecv.n_features_)


In [ ]:
import matplotlib as plt
from sklearn.ensemble import RandomForestRegressor


# define the model
model = RandomForestRegressor()
# fit the model
model.fit(X, y)
# get importance
importance = model.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
 print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

#### RFE

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

# create pipeline
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=5)
model = DecisionTreeClassifier()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
from sklearn.svm import SVC
from sklearn.datasets import make_classification

from yellowbrick.model_selection import RFECV

# Instantiate RFECV visualizer with a linear SVM classifier
visualizer = RFECV(SVC(kernel='linear', C=1))

visualizer.fit(X, y)        # Fit the data to the visualizer
visualizer.show()           # Finalize and render the figure

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from yellowbrick.model_selection import rfecv
v = StratifiedKFold(5)
visualizer = rfecv(RandomForestClassifier(), X=X, y=y, scoring='f1_weighted')
visualizer.show()   

In [ ]:
from sklearn.linear_model import LogisticRegression
columns=new.columns
# define the model
model = LogisticRegression()
# fit the model
model.fit(X, y)
# get importance prediction 
importance = model.coef_[0]
# summarize feature importance
for i,v in enumerate(importance):
 print('Feature: %s, Score: %.5f' % (columns[i],v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
estimator = SVR(kernel="linear")
selector = RFE(estimator, n_features_to_select=88, step=1)
selector = selector.fit(X, y)
selector.support_
selector.ranking_